#### Libraries I used

In [1]:
import httplib
import urllib2
import random
import re
import requests
from httplib import IncompleteRead
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sys  
reload(sys)  
sys.setdefaultencoding('latin-1')

### Load stored data:

In [3]:
all_page=pd.read_csv("all_page.csv",sep=";")
all_page=list(all_page.link)
all_page1=pd.read_csv("base_links.csv",sep=";")
all_page1=list(all_page1.link)

###  Iteration 2: Continuously loop against links obtained at [iteration1](https://github.com/RanaivosonHerimanitra/Flood-scraping/blob/master/iteration1.ipynb) to attain archives



#### randomly sample to introduce randomness

In [28]:
all_page= random.sample(all_page,len(all_page))

In [29]:
for i,s in enumerate(all_page):
    print i, s
    try:
        if "pdf" not in s and "jpg" not in s and "javascript" not in s:
            req = urllib2.Request(s.decode('utf-8'), headers={'User-Agent' : "Magic Browser"})
            page = urllib2.urlopen( req )
        else:
            continue
    except urllib2.HTTPError,e:
        continue
    except urllib2.URLError, e:
        continue
    except httplib.HTTPException, e:
        continue
    except Exception:
        continue
    #handle page that does not actually exist
    mypage=page.read()
    if len(mypage)==0:
        print "the above page does not exist"
        continue
    else:
        soup = BeautifulSoup(mypage)
        #find all links from the current "page"
        all_links= soup.find_all("a")
        #ensure all links are valid
        link_inside= [link.get("href") for link in all_links if link.get("href") is not None]
        link_inside= [link if link.startswith('http') or link.startswith('www') else s + link for link in link_inside]
        #do not append links from starter links
        link_inside = [link for link in link_inside if link not in site_web]
        #do not append links from previous records
        link_inside = [link for link in link_inside if link not in all_page]
        #do not append links in the current records
        link_inside = [link for link in link_inside if link not in all_page1]
        for k in np.unique(link_inside):
            print k, len(all_page1)
            all_page1.append(k)
    #save on the fly
    df = pd.DataFrame(data=all_page1, index= range(len(all_page1)), columns=['link'])
    df.to_csv("base_links.csv",sep=";",encoding="utf-8")

KeyboardInterrupt: 

### Keep unique pages

In [30]:
all_page1=np.unique(all_page1)
all_page1=list(all_page1)
len(all_page1)

92197

### Filter to remove social media links

In [31]:
f=filter(lambda x: 'linkedin' in x or 'facebook' in x or 'twitter' in x or 'youtube' in x or 'http://www.meteo-senegal.net' in x, all_page1)
for k in site_web:
    f.append(k)
all_page1= [x for x in all_page1 if x not in f]
len(all_page1)

91875

### Load data (base links) again

In [32]:
all_page1=pd.read_csv("base_links.csv",sep=";")
all_page1=list(all_page1.link)
all_page=pd.read_csv("all_page.csv",sep=";")
all_page=list(all_page.link)

### Search for keywords related to flood events

In [33]:
flood_kwds='(meteo|evacués|evacuation|SOS|crise humanitaire|sinistres|précipitations|pluviométri|bidonville|'
flood_kwds=flood_kwds+'montée des eaux|flood|zone risque|remblayage|effondrement|engorgement|débordé|digue'
flood_kwds=flood_kwds+'|submergé|éclatement barrage|rupture barrage|glissement terrain|climatique|inondé|inondables|'
flood_kwds= flood_kwds+'inondation|inondations|pluies|pluie|banlieu|bas-quartiers)'
#2nd iter
flood_page_from_all_page1= [x for x in all_page1 if re.search(flood_kwds,x) is not None]
#1st iter
flood_page_from_all_page= [x for x in all_page if re.search(flood_kwds,x) is not None]
flood_page=flood_page_from_all_page + flood_page_from_all_page1
flood_page=np.unique(flood_page)
len(flood_page)

262

## Save results ==>262 links

In [34]:
df = pd.DataFrame(data=flood_page, index= range(len(flood_page)), columns=['link'])
df.to_csv("potential_flood_events.csv",sep=";",encoding="utf-8")
df.shape

(262, 1)